# 🎯 베이스라인 구현 - KoBART 기반 대화 요약
> PRD 계획에 따른 베이스라인 모델 구현

**목표 성능**: ROUGE-F1 47+

In [1]:
# 환경 설정
import sys
import os
from pathlib import Path

# 프로젝트 루트 경로 추가
notebook_dir = Path.cwd()
# notebooks/team/CHH -> notebooks -> team -> natural-language-processing-competition
project_root = notebook_dir.parent.parent.parent  # 3번만 parent 사용!

# 다른 프로젝트 경로 제거하고 현재 프로젝트 경로만 추가
sys.path = [p for p in sys.path if 'computer-vision-competition' not in p]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project Root: {project_root}")
print(f"Current Dir: {notebook_dir}")

# 필요한 라이브러리 임포트
import yaml
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from transformers import AutoTokenizer, BartForConditionalGeneration
from rouge import Rouge
import wandb

# 커스텀 모듈 임포트
from src.logging.notebook_logger import NotebookLogger
from src.utils.gpu_optimization.team_gpu_check import check_gpu_tier

print("Libraries imported successfully!")

Project Root: /home/ieyeppo/AI_Lab/natural-language-processing-competition
Current Dir: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH
Libraries imported successfully!


In [2]:
# 설정 로드
config_path = notebook_dir / 'configs' / 'config_baseline.yaml'
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print(f"Model: {config['model']['name']}")
print(f"Batch Size: {config['training']['batch_size']}")

Model: digit82/kobart-summarization
Batch Size: 16


In [ ]:
# 로거 초기화
# config의 로그 경로 사용
def get_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

log_dir = get_path(config['paths']['log_dir'])
log_dir.mkdir(parents=True, exist_ok=True)

logger = NotebookLogger(
    log_path=str(log_dir / 'baseline.log'),
    print_also=True
)
logger.write('=== Baseline Experiment Started ===')

In [4]:
# GPU 체크
if torch.cuda.is_available():
    gpu_tier = check_gpu_tier()
    logger.write(f"GPU Tier: {gpu_tier}")
    logger.write(f"GPU: {torch.cuda.get_device_name(0)}")

GPU Tier: LOW
GPU: NVIDIA GeForce RTX 4090


In [ ]:
# 데이터 경로 설정 및 로드
# config 파일의 경로 사용
def get_data_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

# config에서 데이터 경로 가져오기
train_path = get_data_path(config['paths']['train_file'])
dev_path = get_data_path(config['paths']['dev_file'])
test_path = get_data_path(config['paths']['test_file'])

logger.write(f"Loading data from config paths:")
logger.write(f"  - Train: {train_path}")
logger.write(f"  - Dev: {dev_path}")
logger.write(f"  - Test: {test_path}")

# 데이터 로드
train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

logger.write(f"Data loaded successfully!")
logger.write(f"Train samples: {len(train_df)}")
logger.write(f"Dev samples: {len(dev_df)}")
logger.write(f"Test samples: {len(test_df)}")

# 데이터 샘플 출력
print("\nSample data:")
print(train_df.head(2))

In [6]:
# WandB 초기화
wandb.init(
    project=config['wandb']['project'],
    entity=config['wandb']['entity'],
    name=config['wandb']['name'],
    config=config
)

wandb: Currently logged in as: ieyeppo-job (kimsunmin0227-hufs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(config['model']['name'])
model = BartForConditionalGeneration.from_pretrained(config['model']['name'])

logger.write(f"Model loaded: {config['model']['name']}")

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


Model loaded: digit82/kobart-summarization


## 학습 코드는 config 파일 설정에 따라 구현